In [63]:
## Import dependencies - pyscopg2 is a Python-PostgreSQL database adapter 

import psycopg2
import pandas as pd

In [64]:
# Establish connection to PostgreSQL database 

conn = psycopg2.connect(
    host="localhost",
    database="user_session",
    user="postgres",
    password="AAA009wn73ed")

conn.autocommit = True

In [65]:
# The curosor class allows Python code to execute PostgreSQL commands in a database session

cursor = conn.cursor()

In [66]:
# Create SQL query to select all data from an existing PostgreSQL table within the user_session database

selectQuery = ('''SELECT * FROM customer_session_data''')

# Execute the database operation through cursor.execute 

cursor.execute(selectQuery)

# cursor.fetchall returns all the records within the table, assign results to DataFrame & set column headers

userSession_Data =  pd.DataFrame(cursor.fetchall(), 
                                 columns=['Session Timestamp', 'Event Type', 'Product ID', 
                                          'Category ID', 'Category Code', 'Brand', 
                                          'Price', 'User ID', 'User Session ID'])

# Print first 5 rows of DataFrame by using ['DataFrame'].head() to check the data 

userSession_Data.head()

,Session Timestamp,Event Type,Product ID,Category ID,Category Code,Brand,Price,User ID,User Session ID
0,2019-01-10,view,44600062,2.10381E+18,None,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-01-10,view,3900821,2.05301E+18,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-01-10,view,17200506,2.05301E+18,furniture.living_room.sofa,None,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-01-10,view,1307067,2.05301E+18,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-01-10,view,1004237,2.05301E+18,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [67]:
# Check number of rows & columns by using ['DataFrame'].shape method:

# 1,048,575 ROWS
# 9 COLUMNS 

userSession_Data.shape

(1048575, 9)

In [68]:
# Check data types in each column of the DataFrame with the ['DataFrame'].dtypes method 

userSession_Data.dtypes

Session Timestamp    datetime64[ns]
Event Type                   object
Product ID                    int64
Category ID                  object
Category Code                object
Brand                        object
Price                       float64
User ID                       int64
User Session ID              object
dtype: object

In [69]:
# Change format of the Session Timestamp column to include H, M & S timeframes as the original format included Y, M & D

userSession_Data['Session Timestamp'] = pd.to_datetime(userSession_Data['Session Timestamp'],
                                                format="%Y-%m-%d %H:%M:%S")

# Print DataFrame to vaidate change in date format 

userSession_Data

,Session Timestamp,Event Type,Product ID,Category ID,Category Code,Brand,Price,User ID,User Session ID
0,2019-01-10 00:00:00,view,44600062,2.10381E+18,None,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-01-10 00:00:00,view,3900821,2.05301E+18,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-01-10 00:00:00,view,17200506,2.05301E+18,furniture.living_room.sofa,None,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-01-10 00:00:00,view,1307067,2.05301E+18,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-01-10 00:00:00,view,1004237,2.05301E+18,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...,...,...,...,...
1048570,2019-01-10 17:27:00,view,1005105,2.05301E+18,electronics.smartphone,apple,1415.48,537482499,60b0e052-920c-4469-9627-952aa88d0b16
1048571,2019-01-10 17:27:00,view,2601292,2.05301E+18,None,gefest,47.62,553278643,97ca518d-44df-4fca-8081-4d2d95b85607
1048572,2019-01-10 17:27:00,view,3601241,2.05301E+18,appliances.kitchen.washer,lg,350.70,552637214,18f1b24c-dae4-4d7a-a470-3f78c6b15533
1048573,2019-01-10 17:27:00,view,1004754,2.05301E+18,electronics.smartphone,honor,257.38,542200836,fe582251-252a-4b79-af15-7c5c5ce8c6f1


In [70]:
## Split Category Code column into multiple columns based on '.' delimiter. 
# There are multiple levels of category within the data, but they are separated by full stops in the Category Code column
# Splitting the column into multiple category levels increases granularity of the data 


# Create 3 new columns to hold each Category Code Level & split the Category Code column based on the '.' delimiter
# Where Category Codes do not have additional categories, 'None' is returned 

userSession_Data[['Category Code L2', 'Category Code L3', 'Category Code L4']] = userSession_Data['Category Code'].str.split('.', n=2, expand=True)

# Print results 
userSession_Data.head()

,Session Timestamp,Event Type,Product ID,Category ID,Category Code,Brand,Price,User ID,User Session ID,Category Code L2,Category Code L3,Category Code L4
0,2019-01-10,view,44600062,2.10381E+18,None,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,None,None,None
1,2019-01-10,view,3900821,2.05301E+18,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,appliances,environment,water_heater
2,2019-01-10,view,17200506,2.05301E+18,furniture.living_room.sofa,None,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,furniture,living_room,sofa
3,2019-01-10,view,1307067,2.05301E+18,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,computers,notebook,None
4,2019-01-10,view,1004237,2.05301E+18,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,electronics,smartphone,None


In [72]:


userSession_Data = userSession_Data.apply(lambda x: x.astype(str).str.capitalize())

userSession_Data

,Session Timestamp,Event Type,Product ID,Category ID,Category Code,Brand,Price,User ID,User Session ID,Category Code L2,Category Code L3,Category Code L4
0,2019-01-10 00:00:00,View,44600062,2.10381e+18,Nan,Shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,Nan,Nan,Nan
1,2019-01-10 00:00:00,View,3900821,2.05301e+18,Appliances.environment.water_heater,Aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,Appliances,Environment,Water_heater
2,2019-01-10 00:00:00,View,17200506,2.05301e+18,Furniture.living_room.sofa,Nan,543.1,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,Furniture,Living_room,Sofa
3,2019-01-10 00:00:00,View,1307067,2.05301e+18,Computers.notebook,Lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,Computers,Notebook,Nan
4,2019-01-10 00:00:00,View,1004237,2.05301e+18,Electronics.smartphone,Apple,1081.98,535871217,C6bd7419-2748-4c56-95b4-8cec9ff8b80d,Electronics,Smartphone,Nan
...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2019-01-10 17:27:00,View,1005105,2.05301e+18,Electronics.smartphone,Apple,1415.48,537482499,60b0e052-920c-4469-9627-952aa88d0b16,Electronics,Smartphone,Nan
1048571,2019-01-10 17:27:00,View,2601292,2.05301e+18,Nan,Gefest,47.62,553278643,97ca518d-44df-4fca-8081-4d2d95b85607,Nan,Nan,Nan
1048572,2019-01-10 17:27:00,View,3601241,2.05301e+18,Appliances.kitchen.washer,Lg,350.7,552637214,18f1b24c-dae4-4d7a-a470-3f78c6b15533,Appliances,Kitchen,Washer
1048573,2019-01-10 17:27:00,View,1004754,2.05301e+18,Electronics.smartphone,Honor,257.38,542200836,Fe582251-252a-4b79-af15-7c5c5ce8c6f1,Electronics,Smartphone,Nan


In [73]:
## Calculate times of the day with high and low relative user session volume 

# Count the number of events per each timeframe with the value_counts function, referencing Session Timestamp column
timeData = pd.DataFrame(userSession_Data.value_counts('Session Timestamp'), 
                  columns=['Count of Events']).reset_index()

# Sort data based on earliest session timestamp (00:00:00)
timeData.sort_values(by='Session Timestamp', inplace=True)

# Print DataFrame and sort descending by Count of Events column
timeData.sort_values(by='Count of Events', ascending=False)

,Session Timestamp,Count of Events
0,2019-01-10 16:42:00,1756
1,2019-01-10 16:36:00,1749
2,2019-01-10 16:23:00,1736
3,2019-01-10 16:18:00,1727
4,2019-01-10 16:43:00,1718
...,...,...
1019,2019-01-10 00:31:00,1
1023,2019-01-10 01:07:00,1
1021,2019-01-10 00:28:00,1
1022,2019-01-10 00:27:00,1


In [11]:
## Calculate instances of each Event Type: View, Purchase & Cart 
# This data can be used to calculate the overall conversion rate of the website

# Create new DataFrame and use the .value_counts method to count the instances of each unique Event Type
eventTypes = pd.DataFrame(userSession_Data.value_counts('Event Type'), columns=['Count of Events']).reset_index()

# The Event Types were in lower case, so str.capitalize is used to tidy the text data & amend to capital for first letter 
eventTypes['Event Type'] = eventTypes['Event Type'].str.capitalize()

# Print results 
eventTypes

,Event Type,Count of Events
0,View,1016239
1,Purchase,17296
2,Cart,15040


In [12]:
## Calculate total number of events (all categories) per brand

brandVolume = pd.DataFrame(userSession_Data.value_counts('Brand'), columns=['Count of Events']).reset_index()

# Capitalize first letter of Brand data & Event Type data for tidiness 

brandVolume['Brand'] = brandVolume['Brand'].str.capitalize()

brandVolume

,Brand,Count of Events
0,Samsung,126027
1,Apple,107634
2,Xiaomi,72507
3,Huawei,29360
4,Lucente,17799
...,...,...
2240,Robobloq,1
2241,Lava,1
2242,Floresan,1
2243,Zarkoperfume,1


In [35]:
## Calculate brands with the highest purchase volume 

# groupby Brand and Event Type and use size() which returns the number of elements within an object 
# Create a new DataFrame which will show the count of each event type per unique brand name

brandEventData = pd.DataFrame(userSession_Data.groupby(["Brand", "Event Type"]).size(), columns=['Count of Events']).reset_index()

## Filter the Event Type column for 'purchase' using .loc, assign to variable named brandPurchaseData

brandPurchaseData = brandEventData.loc[brandEventData['Event Type'] == 'purchase'] 

# Sort DataFrame from highest number of purchase events to lowest, print results 

brandPurchaseData.sort_values(by='Count of Events', ascending=False)

,Brand,Event Type,Count of Events
2391,samsung,purchase,4053
149,apple,purchase,3565
2965,xiaomi,purchase,1337
1271,huawei,purchase,617
1642,lucente,purchase,307
...,...,...,...
1003,forza,purchase,1
2165,polti,purchase,1
1001,forward,purchase,1
996,forlux,purchase,1


In [28]:
# Count the instances of each category code by using the value_counts() method
# This will return the count of instances for each unique category code 

categoryCodes = pd.DataFrame(userSession_Data['Category Code'].value_counts()).reset_index()

# Rename columns following the index reset, as by default it will be renamed 'index' 

categoryCodes.columns = ['Category L1', 'Count of Events']

# Extract top 100 category codes, using nlargest & assigning 100 to 'n' - This number can be amended if required
# The below DataFrame displays the best performing category codes in terms of event count

topCategoryCodes = categoryCodes.nlargest(n=100, columns=['Count of Events'])

topCategoryCodes

,Category L1,Count of Events
0,electronics.smartphone,286986
1,electronics.clocks,37264
2,computers.notebook,30108
3,electronics.audio.headphone,27567
4,electronics.video.tv,23216
...,...,...
95,apparel.shoes.sandals,307
96,apparel.trousers,290
97,country_yard.lawn_mower,270
98,appliances.kitchen.coffee_grinder,259
